---
# Markov Decision Process (MDP) and Reinforcement Learning (RL) Theory
---

## Purpose

The purpose of this workshop is to provide practical experience working with Markov Decision Processes (MDPs) and Reinforcement Learning (RL), specifically Value Iteration and Q-Learning. In addition, we interpret the Bellman equations and the Expectimax search tree.

## Description

The workshop is designed as a walk-through, where we reason our way toward the correct answers. Read through the workshop on your own, and ask the instructor if you have any questions.

---
## Contents
---

This notebook contains the following sections:

- [1. Interpreting an MDP Problem](#1-interpreting-an-mdp-problem), which covers how to interpret an MDP problem.
- [2. The V-Value Function](#2-the-v-value-function), which introduces the V-value function.
- [3. The Q-Value Function](#3-the-q-value-function), which introduces the Q-value function.
- [4. The Bellman Equations and the Expectimax Search Tree](#4-the-bellman-equations-and-the-expectimax-search-tree), which covers the Bellman equations.
- [5. Value Iteration](#5-value-iteration), which covers the value interation algorithm.
- [6. Q-Learning](#6-q-learning), which covers the q-learning algorithm.

---
## 1. Interpreting an MDP Problem

Suppose we have an MDP problem defined as follows:
- Note that $s'$ is an abbreviation for the *next state*, and $a'$ is the action taken in that *next state*.

State space:

$$
s_i \in S, \text{ where } S = \{s_1,s_2,s_3,s_4,s_5,s_6,s_7\}, \text{ i.e. 7 states }
$$

Action space:

$$
a_i \in A(s), \text{ where } A(s) = \{a_1 = left, a_2 = right\}, \text{ i.e. 2 actions per state } s
$$

Start state:

$$
s_0 = s_4
$$

Transition function:

$$
T(s,a,s´) = 
\left\{
\begin{array}{ll}
0.9, & \text{ if } (s=s_i,a=a_1,s'=s_{i-1}) \text{ or } (s=s_i,a=a_2,s'=s_{i+1})\\
1.0, & \text{ if } s \in \{s_1,s_7\} \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
0, & \text{ otherwise }
\end{array}
\right\}
$$

Reward function:

$$
R(s,a,s´) = 
\left\{
\begin{array}{ll}
-1, & \text{ if } s=s_1 \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
+1, & \text{ if } s=s_7 \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
0, & \text{ otherwise }
\end{array}
\right\}
$$

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |

</div>
<div>

||
|-----|
|$\quad E \quad$|

</div>
</div>

We therefore have:
- 7 states $\{s_1,s_2,s_3,s_4,s_5,s_6,s_7\}$
- 2 actions $\{a_1,a_2\}$ in each state, where $a_1=left$ and $a_2=right$.
- Our agent starts in state $s_4$.

The transition function states that:
- With 90% probability the agent moves to the intended next state, i.e.:
  - If the agent chooses to move left, it moves to the state immediately to the left of the current state with probability 0.9.
  - If the agent chooses to move right, it moves to the state immediately to the right of the current state with probability 0.9.
- However, there is a 10% probability of the agent **not** ending up in the intended next state (moving in the  **opposite** of the intended direction), i.e.:
  - If the agent chooses to move left, it may actually move right instead with probability 0.1.
  - If the agent chooses to move right, it may actually move left instead with probability 0.1.
- Finally, if the agent is in state $s_1$ or $s_7$, there is a 100% probability that the agent is transitioned to the virtual state $E$ regardless of action.
  - Therefore, $s_1$ and $s_7$ are terminal states, but the reward is only given when the agent takes an action in that state ($s_1$ or $s_7$) and transitions into $E$.
  - Futhermore, there are no legal actions in the virtual state $E$.

The reward function states that:
- The agent receives the reward $–1$ when it executes any action in the terminal state $s_1$ (and therefore ends up in the virtual state $E$).
- The agent receives the reward $+1$ when it executes any action in terminal state $s_7$ (and therefore ends up in the virtual state $E$).
- All other transitions give a reward of 0.

So, in every state, the agent can choose to go $left$ or $right$ (and no actions are available in $E$):

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
| $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ |

</div>
<div>

||
|-----|
|$\quad E \quad$|

</div>
</div>

---
## 2. The V-Value Function

The V-value function (a.k.a. *state-value function*) $V(s)$ for the MDP above can be represented with an array as follows.

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$\,V(s_1)$|$\,V(s_2)$|$\,V(s_3)$|$\,V(s_4)$|$\,V(s_5)$|$\,V(s_6)$|$\,V(s_7)$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$\,V(E)$|

</div>
</div>

The V-value of a state is the *expected return* (i.e. *total discounted reward*) from that state.

The V-value of the virtual state $E$ is always $V(E) = 0$.

**Note!** The *value* of a state is **not** the *immediate reward* for that state.

We can also represent the V-value function with the table below.

|State|V(state)|
|-----|--------|
|$s_1$|$V(s_1)$|
|$s_2$|$V(s_2)$|
|$s_3$|$V(s_3)$|
|$s_4$|$V(s_4)$|
|$s_5$|$V(s_5)$|
|$s_6$|$V(s_6)$|
|$s_7$|$V(s_7)$|
|$E$|$V(E)$|

---
## 3. The Q-Value Function

We can represent the actions we can execute in each state as below.
- Note that there are no actions in the virtual state $E$.

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$a_1 \; \| \; a_2$|$a_1 \; \| \; a_2$|$a_1 \; \| \; a_2$|$a_1 \; \| \; a_2$|$a_1 \; \| \; a_2$|$a_1 \; \| \; a_2$|$a_1 \; \| \; a_2$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$\quad$|

</div>
</div>

The Q-value function (a.k.a. *state-action-value function*) $Q(s,s)$ for the MDP above can be represented with an 2-dimensional array as follows.

<div style="display: flex; gap: 20px; font-size: 55%;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad \quad \quad s_1$ | $\quad \quad \quad s_2$ | $\quad \quad \quad s_3$ | $\quad \quad \quad s_4$ | $\quad \quad \quad s_5$ | $\quad \quad \quad s_6$ | $\quad \quad \quad s_7$ |
|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
$Q(E,a)$|

</div>
</div>

The Q-value of a (state,action)-pair is the *expected return* (i.e. *total discounted reward*) when the action is executed in that state.

The Q-value for any action executed in the virtual state $E$ is always $Q(E,a)=0$.
- Since there aren't any legal actions $a$ i the virtual state $E$, the Q-value is always 0, irrespective of action $a$.

**Note!** The *value* of a (state,action)-pair is **not** the *immediate reward* for that action in that state.

We can also represent the Q-value function with the table below.

|State/Action|$a_1$|$a_2$|
|------------|-----|-----|
|$s_1$|$Q(s_1,a_1)$|$Q(s_1,a_2)$|
|$s_2$|$Q(s_2,a_1)$|$Q(s_2,a_2)$|
|$s_3$|$Q(s_3,a_1)$|$Q(s_3,a_2)$|
|$s_4$|$Q(s_4,a_1)$|$Q(s_4,a_2)$|
|$s_5$|$Q(s_5,a_1)$|$Q(s_5,a_2)$|
|$s_6$|$Q(s_6,a_1)$|$Q(s_6,a_2)$|
|$s_7$|$Q(s_7,a_1)$|$Q(s_7,a_2)$|
|$E$|$Q(E,a_1)$|$Q(E,a_2)$|

---
## 4. The Bellman Equations and the Expectimax Search Tree

Below we see the Bellman equations and an expectimax search tree.

$$
V(s) = \max_a Q(s,a)
$$

$$
Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]
$$

$$
V(s) = \max_a \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]
$$

<div align="center">
<img src="images/expectimax_search_tree.png">
</div>

If we start by interpreting the expectimax search tree, we see that there is a MAX operation between the top node <span style="color: #87CEFA;">s</span> and its child nodes <span style="color: #99ff99;">(s,a)</span>. That is, there is one child node <span style="color: #99ff99;">(s,a)</span> beneath the node (state) <span style="color: #87CEFA;">s</span> for each valid action <span style="color: #99ff99;">a</span> in state <span style="color: #87CEFA;">s</span>, and we simply want to choose the child node with the highest value <span style="color: #99ff99;">Q(s,a)</span>.

Suppose we had two valid actions $a_1$ and $a_2$ under state $s$, then the MAX operation tells us that we should choose the child node with the largest value $Q(s,a_1)$ or $Q(s,a_2)$, i.e. $\max_{a \in \{a_1,a_2\}} Q(s,a)$, or simply $max\left(Q(s,a_1),Q(s,a_2)\right)$.

For example, assume we have the two possible actions $a_1$ and $a_2$ in state $s$, with:
- $Q(s,a_1)=1.0$
- $Q(s,a_2)=2.0$

Then the value of the state $s$ is:

$$
\begin{aligned}
V(s) &= \max_{a \in \{a_1,a_2\}} Q(s,a) \\
     &= max\left(Q(s,a_1), Q(s,a_2)\right) \\
     &= max(1.0, 2.0) \\
     &= 2.0
\end{aligned}
$$

So we would have chosen the child node $(s,a_2)$ with the value $Q(s,a_2)=2.0$, which is the value of the topmost node (state) $s$ in the expectimax search tree. This is exactly what the first Bellman equation $V(s) = \max_a Q(s,a)$ tells us.

Now, what happens if we instead want to compute the value of a Q-state, i.e., the value of a <span style="color: #99ff99;">(s,a)</span> node, i.e. <span style="color: #99ff99;">Q(s,a)</span>? In the expectimax search tree, we see that there is an EXPECTATION operation, i.e., $\mathbb{E}_{s' \sim P(\cdot \mid s,a)}\![R(s,a,s') + \gamma V(s')]$, between the node <span style="color: #99ff99;">(s,a)</span> and the child (successor) nodes <span style="color: #87CEFA;">s'</span>. So, there is one child node <span style="color: #87CEFA;">s'</span> under the parent node <span style="color: #99ff99;">(s,a)</span> for each possible next state (node) <span style="color: #87CEFA;">s'</span>, and we want to compute a weighted average over their values, where each child node has the probability (weight) $T(s,a,s')$ and the value $R(s,a,s') + \gamma V(s')$. So the Q-value is $Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]$.

Suppose we have two possible next states $s_1'$ and $s_2'$. The EXPECTATION operation tells us to compute the weighted value of each child node, i.e. $T(s,a,s_1') \cdot [R(s,a,s_1') + \gamma \cdot V(s_1')]$ and $T(s,a,s_2') \cdot [R(s,a,s_2') + \gamma \cdot V(s_2')]$, and then to add them together, i.e. $T(s,a,s_1') \cdot [R(s,a,s_1') + \gamma \cdot V(s_1')] + T(s,a,s_2') \cdot [R(s,a,s_2') + \gamma \cdot V(s_2')]$, or simply $\sum_{s' \in \{s_1',s_2'\}} T(s,a,s') \cdot [R(s,a,s') + \gamma \cdot V(s')]$.

For example, assume we have the two possible next states $s_1'$ and $s_2'$, with:
- $T(s,a,s_1')=0.1$
- $R(s,a,s_1')=0.1$
- $V(s_1')=1.0$
- $T(s,a,s_2')=0.9$
- $R(s,a,s_2')=0.2$
- $V(s_2')=2.0$
- $\gamma=0.9$

Then:

$$
\begin{aligned}
Q(s,a) &= \mathbb{E}\!\left[R(s,a,s') + \gamma V(s')\right] \\
       &= \mathbb{E}_{s' \sim P(\cdot \mid s,a)}\![R(s,a,s') + \gamma V(s')] \\
       &= \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')] \\
       &= T(s,a,s_1') \cdot [R(s,a,s_1') + \gamma \cdot V(s_1')] + T(s,a,s_2') \cdot [R(s,a,s_2') + \gamma \cdot V(s_2')] \\
       &= 0.1 \cdot [0.1 + 0.9 \cdot 1.0] + 0.9 \cdot [0.2 + 0.9 \cdot 2.0] \\
       &= 0.1 + 1.8 \\
       &= 1.9
\end{aligned}
$$

So the Q-value for node $(s,a)$ is $Q(s,a)=1.9$.

This is exactly what the second Bellman equation $Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]$ expresses.

If we substitute $Q(s,a)$ on the right-hand side of the first Bellman equation $V(s) = \max_a Q(s,a)$ with the right-hand side of the second Bellman equation $Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]$, we obtain the third Bellman equation $V(s) = \max_a \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]$.

This equation tells us that we can compute the value $V(s)$ of a state $s$ by taking, for each possible action $a$ in state $s$, the weighted sum of all possible successor states $s'$ that could be reached by performing action $a$ in state $s$, and then selecting the largest (MAX) of these weighted sums across all actions $a$. In other words, this equation allows us to compute $V(s)$ from the values of successor states $V(s')$ without needing to explicitly compute $Q(s,a)$.

Can we do the same thing for Q-values? That is, can we compute a Q-value $Q(s,a)$ from successor Q-values $Q(s',a')$, without needing to explicitly compute the V-values $V(s')$?

So, we want to compute the value of the second Q-node $Q(s,a)$ in the expectimax search tree directly from values of nodes $Q(s',a')$ furthest down in the tree, without first computing the values of nodes $V(s')$ next-to-last in the tree.

Starting from the second Bellman equation $Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]$, we see that it allows us to compute a Q-value $Q(s,a)$ from successor V-values $V(s')$ in the tree.

But from the first Bellman equation $V(s) = \max_a Q(s,a)$, we know that we can compute a V-value $V(s)$ by taking the MAX over successor Q-values $Q(s,a)$, i.e. $V(s) = \max_a Q(s,a)$, or equivalently (if we want to compute the value of the state $s'$) $V(s') = \max_{a'} Q(s',a')$.

If we replace $V(s')$ in the second Bellman equation $Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]$ with this expression for $V(s')$, i.e. $V(s') = \max_{a'} Q(s',a')$, we get:

$$
Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma \max_{a'} Q(s',a')]
$$

Thus, we can compute Q-values $Q(s,a)$ directly from successor Q-values $Q(s',a')$ without ever computing V-values $V(s')$ explicitly. This gives us a fourth Bellman equation to add to the previous three:

$$
V(s) = \max_a Q(s,a)
$$

$$
Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]
$$

$$
V(s) = \max_a \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V(s')]
$$

$$
Q(s,a) = \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma \max_{a'} Q(s',a')]
$$

<div align="center">
<img src="images/expectimax_search_tree.png">
</div>

Now we know how to compute V-values from other V-values or Q-values, and how to compute Q-values from other Q-values or V-values.

---
## 5. Value Iteration

The Bellman equations are the mathematical formulation for solving MDP problems. In practice, we solve MDP problems using an iterative solution method (*value iteration* or *policy iteration*).

Let's start with *value iteration*. First we set the V-value to 0 for every state, i.e., $V(s) = 0, \forall_s \in S$, where $S$ is the *state space* (we can call this iteration $k=0$).

Then we perform a **Bellman backup** (i.e., we update all V-values $V(s)$ from successor V-values $V(s')$ for every state $s$ in the same iteration $k$.

We can rewrite the third Bellman equation as an iteration, as bellow, where we also include iteration $k=0$.
- We use $\leftarrow$ to indicate an update (backup) rather than equality $=$.

$$
V_0(s) \leftarrow 0, \forall_s \in S
$$

$$
V_{k+1}(s) \leftarrow \max_a \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V_k(s')]
$$

Now, let us apply this to the MDP problem that we defined earlier, where we use a *discount factor* of $\gamma = 0.9$.

State space:

$$
s_i \in S, \text{ where } S = \{s_1,s_2,s_3,s_4,s_5,s_6,s_7\}, \text{ i.e. 7 states }
$$

Action space:

$$
a_i \in A(s), \text{ where } A(s) = \{a_1 = left, a_2 = right\}, \text{ i.e. 2 actions per state } s
$$

Start state:

$$
s_0 = s_4
$$

Transition function:

$$
T(s,a,s´) = 
\left\{
\begin{array}{ll}
0.9, & \text{ if } (s=s_i,a=a_1,s'=s_{i-1}) \text{ or } (s=s_i,a=a_2,s'=s_{i+1})\\
1.0, & \text{ if } s \in \{s_1,s_7\} \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
0, & \text{ otherwise }
\end{array}
\right\}
$$

Reward function:

$$
R(s,a,s´) = 
\left\{
\begin{array}{ll}
-1, & \text{ if } s=s_1 \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
+1, & \text{ if } s=s_7 \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
0, & \text{ otherwise }
\end{array}
\right\}
$$

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |

</div>
<div>

||
|-----|
|$\quad E \quad$|

</div>
</div>

We start by setting all V-values to 0 in iteration $k = 0$:

$$
V(s) \leftarrow 0, \forall_s \in S
$$

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$\,V(s_1)$|$\,V(s_2)$|$\,V(s_3)$|$\,V(s_4)$|$\,V(s_5)$|$\,V(s_6)$|$\,V(s_7)$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$\,V(E)$|

</div>
</div>

So, our V-values in iteration $k=0$, i.e. $V_0(s)$, will look like this:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$\quad 0 \quad$|

</div>
</div>

Or, in table form:

|State|V(state)|
|-----|--------|
|$s_1$|$0$|
|$s_2$|$0$|
|$s_3$|$0$|
|$s_4$|$0$|
|$s_5$|$0$|
|$s_6$|$0$|
|$s_7$|$0$|
|$E$|$0$|

Then we carry out iteration $k=1$ using:

$$
V_{k+1}(s) \leftarrow \max_a \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V_k(s')]
$$

Here we see that we are filling a **new** array $V_{k+1}(s)$ by using values from the array in the previous iteration $V_k(s)$. These previous V-values are shown in the array above (from iteration $k=0$. We must compute $V_{k+1}(s)$ for every state $s$.

According to the definition of our MDP, we receive reward -1 and +1 if we execute any action in the terminal states $s_1$ and $s_7$ respectively, which transitions us to the virtual state $E$, where the episode ends. For all other actions, the reward is $0$, accoring to our MDP.

If we look at our V-value array above (from iteration $k=0$), we see that if we perform any action in the states $s_2,s_3,s_4,s_5,s_6$, it doesn't matter which next state $s'$ we end up in, since the values of the next states are all $V(s') = 0$, and the immediate rewards for those actions are all $R(s,a,s') = 0$. Therefore, the new V-values for all these states is 0 in iteration $k=1$, i.e. for $s \in \left\{s_2,s_3,s_4,s_5,s_6\right\}$:

$$
\begin{aligned}
V_1(s) &\leftarrow \max_a \sum_{s'} T(s,a,s') [R(s,a,s') + \gamma V_0(s')] \\
       &\leftarrow \max_a \sum_{s'} T(s,a,s') [0 + \gamma \cdot 0] \\
       &\leftarrow 0
\end{aligned}
$$

So, in this iteration, i.e. $k=1$, we only need to compute the V-values for $s \in \{s_1,s_7\}$ (for this MDP problem).

From the definition of our MDP, we also know that it doesn't matter which action we take in the states $s \in \{s_1,s_7\}$, since the transition probability is $T(s,a,s') = 1.0$ that we transition to the next state $s'=E$, where the episode ends (we can also interpret this as if there is only one action $a=x$ available in these two states, which deterministically leads to the next state $s'=E$).

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| <span style="padding: 22px; color: #87CEFA;">s<sub>1</sub></span> | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | <span style="padding: 22px; color: #87CEFA;">s<sub>7</sub></span> |
| $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ |

</div>
<div>

||
|-----|
|$\quad E \quad$|

</div>
</div>

If we plug all of this into the update formula for $s=s_1$, we get:

$$
{\small
\begin{aligned}
V_1(s_1) &\leftarrow \max_a \sum_{s'} T(s_1,a,s') \cdot [R(s_1,a,s') + \gamma \cdot V_0(s')] \\
         &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & T(s_1,x,E) \cdot [R(s_1,x,E) + \gamma \cdot V_0(E)]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & 1.0 \cdot [-1.0 + 0.9 \cdot 0]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & -1.0 \\
          \end{array}
          \right) \\
          &\leftarrow -1.0
\end{aligned}
}
$$

Likewise, for $s=s_7$, we get:

$$
{\small
\begin{aligned}
V_1(s_7) &\leftarrow \max_a \sum_{s'} T(s_7,a,s') \cdot [R(s_7,a,s') + \gamma \cdot V_0(s')] \\
         &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & T(s_7,x,E) \cdot [R(s_7,x,E) + \gamma \cdot V_0(E)]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & 1.0 \cdot [1.0 + 0.9 \cdot 0]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & 1.0 \\
          \end{array}
          \right) \\
          &\leftarrow 1.0
\end{aligned}
}
$$

After iteration $k=1$, our V-value array looks as follows:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$\quad -1 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 1 \quad$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$\quad 0 \quad$|

</div>
</div>

Or, in table form:

|State|V(state)|
|-----|--------|
|$s_1$|$-1$|
|$s_2$|$0$|
|$s_3$|$0$|
|$s_4$|$0$|
|$s_5$|$0$|
|$s_6$|$0$|
|$s_7$|$1$|
|$E$|$0$|

In iteration $k=2$, we use the values from the array above (from iteration $K=1$) to compute the V-values in iteration $k=2$.

Using the same reasoning as before, we see that the states $s \in \{s_3,s_4,s_5\}$ can only lead to successor states with a value of $V(s') = 0$, with reward $R(s,a,s') = 0$, which means their new V-values must also be 0.

However, in the states $s \in \{s_2,s_6\}$, we can perform two different actions:
- $a_1 = left$
- $a_2 = right$

Additionally, the transitions are stochastic (probabilistic), i.e. there is a 90% probability of ending up in the intended next state and a 10% probability of ending up in the unintended next state (opposite direction).

If we perform action $a_1 = left$ in state $s_2$, the following transitions can occur:
- $(s_2,a_1,s_1)$
- $(s_2,a_1,s_3)$

This implies we may end up in either $s_1$ or $s_3$.

Likewise, if we perform action $a_2 = right$ in state $s_2$, the possible transitions are:
- $(s_2,a_2,s_3)$
- $(s_2,a_2,s_1)$

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | <span style="padding: 22px; color: #87CEFA;">s<sub>2</sub></span> | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | <span style="padding: 22px; color: #87CEFA;">s<sub>6</sub></span> | $\quad s_7 \quad$ |
| $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ | $\leftarrow \;\;\; \rightarrow$ |

</div>
<div>

||
|-----|
|$\quad E \quad$|

</div>
</div>

If we plug all of this into the update formula for $s=s_2$, we get:

$$
{\small
\begin{aligned}
V_2(s_2) &\leftarrow \max_a \sum_{s'} T(s_2,a,s') \cdot [R(s_2,a,s') + \gamma \cdot V_1(s')] \\
         &\leftarrow max\left(
          \begin{array}{ll}
          a=a_1: & T(s_2,a_1,s_1) \cdot [R(s_2,a_1,s_1) + \gamma \cdot V_1(s_1)] + T(s_2,a_1,s_3) \cdot [R(s_2,a_1,s_3) + \gamma \cdot V_1(s_3)] \\
          a=a_2: & T(s_2,a_2,s_3) \cdot [R(s_2,a_2,s_3) + \gamma \cdot V_1(s_3)] + T(s_2,a_2,s_1) \cdot [R(s_2,a_2,s_1) + \gamma \cdot V_1(s_1)]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=a_1: & 0.9 \cdot [0 + 0.9 \cdot (-1)] + 0.1 \cdot [0 + 0.9 \cdot 0] \\
          a=a_2: & 0.9 \cdot [0 + 0.9 \cdot 0] + 0.1 \cdot [0 + 0.9 \cdot (-1)]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=a_1: & -0.81 \\
          a=a_2: & -0.09
          \end{array}
          \right) \\
          &\leftarrow -0.09
\end{aligned}
}
$$

Likewise, for $s=s_6$, we get:

$$
{\small
\begin{aligned}
V_2(s_6) &\leftarrow \max_a \sum_{s'} T(s_6,a,s') \cdot [R(s_6,a,s') + \gamma \cdot V_1(s')] \\
         &\leftarrow max\left(
          \begin{array}{ll}
          a=a_1: & T(s_6,a_1,s_5) \cdot [R(s_6,a_1,s_5) + \gamma \cdot V_1(s_5)] + T(s_6,a_1,s_7) \cdot [R(s_6,a_1,s_7) + \gamma \cdot V_1(s_7)] \\
          a=a_2: & T(s_6,a_2,s_7) \cdot [R(s_6,a_2,s_7) + \gamma \cdot V_1(s_7)] + T(s_6,a_2,s_5) \cdot [R(s_6,a_2,s_5) + \gamma \cdot V_1(s_5)]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=a_1: & 0.9 \cdot [0 + 0.9 \cdot 0] + 0.1 \cdot [0 + 0.9 \cdot 1] \\
          a=a_2: & 0.9 \cdot [0 + 0.9 \cdot 1] + 0.1 \cdot [0 + 0.9 \cdot 0]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=a_1: & 0.09 \\
          a=a_2: & 0.81
          \end{array}
          \right) \\
          &\leftarrow 0.81
\end{aligned}
}
$$

For $s = s_!$ (with a deterministic action that leads to $s' = E$, we get:

$$
{\small
\begin{aligned}
V_2(s_1) &\leftarrow \max_a \sum_{s'} T(s_1,a,s') \cdot [R(s_1,a,s') + \gamma \cdot V_1(s')] \\
         &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & T(s_1,x,E) \cdot [R(s_1,x,E) + \gamma \cdot V_1(E)]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & 1.0 \cdot [-1.0 + 0.9 \cdot 0]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & -1.0 \\
          \end{array}
          \right) \\
          &\leftarrow -1.0
\end{aligned}
}
$$

Likewise, for $s = s_7$, we get:

$$
{\small
\begin{aligned}
V_2(s_7) &\leftarrow \max_a \sum_{s'} T(s_7,a,s') \cdot [R(s_7,a,s') + \gamma \cdot V_1(s')] \\
         &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & T(s_7,x,E) \cdot [R(s_7,x,E) + \gamma \cdot V_1(E)]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & 1.0 \cdot [1.0 + 0.9 \cdot 0]
          \end{array}
          \right) \\
          &\leftarrow max\left(
          \begin{array}{ll}
          a=x: & 1.0 \\
          \end{array}
          \right) \\
          &\leftarrow 1.0
\end{aligned}
}
$$

Thus, the V-value for the terminal states does not change in iteration $k=2$. The same holds for all iterations after $k=1$.

After iteration $k=2$, our V-value array therefore looks as follows:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$\quad -1 \quad$|$\quad -0.09 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0 \quad$|$\quad 0.81 \quad$|$\quad 1 \quad$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$\quad 0 \quad$|

</div>
</div>

Or, in table form:

|State|V(state)|
|-----|--------|
|$s_1$|$-1$|
|$s_2$|$-0.09$|
|$s_3$|$0$|
|$s_4$|$0$|
|$s_5$|$0$|
|$s_6$|$0.81$|
|$s_7$|$1$|
|$E$|$0$|

In iteration $k=3$, the V-value for state $s_4$ will not change, according to the same reasoning as in the previous iterations. We also know that the V-values for the terminal states $s_1$ and $s_7$ will not change in any future iterations.

However, we need to compute the V-values for the states $s \in \{s_2,s_3,s_5,s_6\}$ in the same way as in iteration $k=2$. And in all iterations after iteration $k=3$, we will need to compute the V-values for the states $s \in \{s_2,s_3,s_4,s_5,s_6\}$ in the same way as in iteration $k=2$.

We can stop iterating when the change in V-values for all states, between two consecutive iterations, is less than a small constant, for example:

$$
\theta = 1 \cdot 10^{-8} = 0.00000001
$$


Alternatively, we can stop after a fixed number of iterations, for example $k=1000$ (this is called *truncated value iteration*, and it doesn't guarantee that the optimal V-values $V^{*}(s)$ have been reached).

---
## 6. Q-Learning

The Q-learning algorithm is an *off-policy* algorithm, meaning the agent can follow any arbitrary policy (for example, a random policy), but it still learns an optimal policy. This is due to the MAX operation in the formula below:

$$
Q(s,a) \leftarrow (1 - \alpha) Q(s,a) + \alpha \left(r + \gamma \max_{a'} Q(s',a')\right)
$$

which is equivalent to:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \left(r + \gamma \max_{a'} Q(s',a') - Q(s,a) \right)
$$

Assume we have the same MDP problem as before, but now we **do not** know $T(s,a,s')$ or $R(s,a,s')$. In such a case, we can use Q-learning to create a *learning* Reinforcement Learning agent. The agent must now **interact with its environment** in order to learn an optimal policy (i.e. optinal actions in each state).

We further assume that the actions are now *deterministic* (they lead to the intended next state with probability $1.0$) and that the *learning rate* is $\alpha = 0.5$. We also assume that the underlying reward function in the environment is the same as before, i.e., $r = R(s,a,s')$, although we do not actually know this in advance. Instead, the agent must interact with the environment to receive rewards. Let's also use a *discount factor* of $\gamma = 0.9$.

So our MDP now looks like this:

State space:

$$
s_i \in S, \text{ where } S = \{s_1,s_2,s_3,s_4,s_5,s_6,s_7\}, \text{ i.e. 7 states }
$$

Action space:

$$
a_i \in A(s), \text{ where } A(s) = \{a_1 = left, a_2 = right\}, \text{ i.e. 2 actions per state } s
$$

Start state:

$$
s_0 = s_4
$$

Transition function:

$$
T(s,a,s´) = 
\left\{
\begin{array}{ll}
1.0, & \text{ if } (s=s_i,a=a_1,s'=s_{i-1}) \text{ or } (s=s_i,a=a_2,s'=s_{i+1})\\
1.0, & \text{ if } s \in \{s_1,s_7\} \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E
\end{array}
\right\}
$$

Reward function:

$$
R(s,a,s´) = 
\left\{
\begin{array}{ll}
-1, & \text{ if } s=s_1 \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
+1, & \text{ if } s=s_7 \text{ and } a \in \{a_1,a_2\} \text{ where } s'=E\\
0, & \text{ otherwise }
\end{array}
\right\}
$$

Furthermore, we have $\gamma = 0.9$ and $\alpha = 0.5$.

Now, let's solve this MDP problem using Q-learning.

We start by setting all Q-values to 0 for all $(state,action)$-pairs, i.e. $Q(s,a) = 0, \forall_{s \in S, a \in A(s)}$, where $S$ is the *state space* and $A(s)$ is the *action space* in state $s$.

<div style="display: flex; gap: 20px; font-size: 55%;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad \quad \quad s_1$ | $\quad \quad \quad s_2$ | $\quad \quad \quad s_3$ | $\quad \quad \quad s_4$ | $\quad \quad \quad s_5$ | $\quad \quad \quad s_6$ | $\quad \quad \quad s_7$ |
|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|$Q(s_1,a_1) \| Q(s_1,a_2)$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
$Q(E,a)$|

</div>
</div>

This gives us:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$ \quad 0 \quad$|

</div>
</div>

Or, in table form.

|State/Action|$a_1$|$a_2$|
|------------|-----|-----|
|$s_1$|$0$|$0$|
|$s_2$|$0$|$0$|
|$s_3$|$0$|$0$|
|$s_4$|$0$|$0$|
|$s_5$|$0$|$0$|
|$s_6$|$0$|$0$|
|$s_7$|$0$|$0$|
|$E$|$0$|$0$|

Now assume that our Q-learning agent starts in state $s=s_4$, and moves to the right to terminal state $s_7$, and then into the terminal absorbing state (the virtual state) $E$. That is, the agent follows the sequence:

$s_4 \rightarrow a_2 \rightarrow s_5 \rightarrow a_2 \rightarrow s_6 \rightarrow a_2 \rightarrow s_7 \rightarrow a_2 \rightarrow E$

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|  |  |  | $\quad \rightarrow$ | $\quad \rightarrow$ | $\quad \rightarrow$ | $\quad \rightarrow$ |

</div>
<div>

||
|-----|
|$\quad E \quad$|

</div>
</div>

What do the Q-value updates look like according to the formula below?

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \cdot \left(r + \gamma \cdot \max_{a'} Q(s',a') - Q(s,a) \right)
$$

We start by substituting all the values for the first transition, i.e. $(s=s_4,a=a_2,s'=s_5,r=0)$.
- This is also called an *experience*.

$$
Q(s_4,a_2) \leftarrow Q(s_4,a_2) + \alpha \cdot \left(0 + \gamma \cdot \max_{a'} Q(s_5,a') - Q(s_4,a_2) \right)
$$

We also know $\gamma = 0.9$ and $\alpha = 0.5$, but what is $\max_{a'} Q(s_5, a')$?

The expression $\max_{a'} Q(s_5, a')$ tells us that we should take the maximum value among all actions $a'$ in the next state $s'=s_5$. If we look at our current Q-value table, we see that both $Q(s_5,a_1) = 0$ and $Q(s_5,a_2) = 0$. Therefore, $\max_{a' \in \{a_1,a_2\}} Q(s_5, a') = 0$.

Now we can substitute $\max_{a'} Q(s_5, a') = 0$, as well as $\gamma = 0.9$ and $\alpha = 0.5$ into the formula.

$$
Q(s_4,a_2) \leftarrow Q(s_4,a_2) + 0.5 \cdot \left(0 + 0.9 \cdot 0 - Q(s_4,a_2) \right)
$$

If we look at our current Q-value table, we also see that the current value of $Q(s_4,a_2) = 0$ . If we substitute this value into the formula on the right-hand side, then all unknowns are known, and we can complete the calculation:

$$
Q(s_4,a_2) \leftarrow 0 + 0.5 \cdot \left(0 + 0.9 \cdot 0 - 0 \right) = 0
$$

We just obtained the update shown below:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$ \quad 0 \quad$|

</div>
</div>

Or, in table form.

|State/Action|$a_1$|$a_2$|
|------------|-----|-----|
|$s_1$|$0$|$0$|
|$s_2$|$0$|$0$|
|$s_3$|$0$|$0$|
|$s_4$|$0$|$\color{SkyBlue}{0}$|
|$s_5$|$0$|$0$|
|$s_6$|$0$|$0$|
|$s_7$|$0$|$0$|
|$E$|$0$|$0$|

The agent is now in state $s_5$. Notice the **update is done to the state we left**, i.e., $s = s_4$, and for the action we performed in that state, i.e., $a=a_2$. That is, for the Q-value $Q(s_4,a_2)$.

If we continue to follow the sequence $s_4 \rightarrow a_2 \rightarrow s_5 \rightarrow a_2 \rightarrow s_6 \rightarrow a_2 \rightarrow s_7 \rightarrow a_2 \rightarrow E$, we notice that all transitions up to and including $s_6 \rightarrow a_2 \rightarrow s_7$ will give similar updates, i.e., with the value 0.

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, 0$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$ \quad 0 \quad$|

</div>
</div>

Or, in table form.

|State/Action|$a_1$|$a_2$|
|------------|-----|-----|
|$s_1$|$0$|$0$|
|$s_2$|$0$|$0$|
|$s_3$|$0$|$0$|
|$s_4$|$0$|$\color{SkyBlue}{0}$|
|$s_5$|$0$|$\color{SkyBlue}{0}$|
|$s_6$|$0$|$\color{SkyBlue}{0}$|
|$s_7$|$0$|$0$|
|$E$|$0$|$0$|

Now the agent is in state $s=s_7$ (the terminal state) and performs the action $a=a_2$, after which the agent ends up in the next state $E$, i.e., the virtual state. What does this update look like?

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \cdot \left(r + \gamma \cdot \max_{a'} Q(s',a') - Q(s,a) \right)
$$

$$
Q(s_7,a_2) \leftarrow Q(s_7,a_2) + 0.5 \cdot \left(1 + 0.9 \cdot \max_{a'} Q(E,a') - Q(s_7,a_2) \right)
$$

According to the definition of our MDP, we get the reward $r = +1$, and we can read off the Q-value $Q(s_7,a_2) = 0$ from our current Q-value table. Also, according to our MDP, we know that there are no valid actions in state $E$, which means that $\max_{a'} Q(E,a') = 0$. If we substitute these values into the formula, we can complete the calculation:

$$
Q(s_7,a_2) \leftarrow 0 + 0.5 \cdot \left(1 + 0.9 \cdot 0 - 0 \right) = 0.5
$$

So, after the first episode and the sequence $s_4 \rightarrow a_2 \rightarrow s_5 \rightarrow a_2 \rightarrow s_6 \rightarrow a_2 \rightarrow s_7 \rightarrow a_2 \rightarrow E$, our Q-value table looks as follows:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad s_6 \quad$ | $\quad s_7 \quad$ |
|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \mid \color{SkyBlue}{0.5}$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$ \quad 0 \quad$|

</div>
</div>

Or, in table form.

|State/Action|$a_1$|$a_2$|
|------------|-----|-----|
|$s_1$|$0$|$0$|
|$s_2$|$0$|$0$|
|$s_3$|$0$|$0$|
|$s_4$|$0$|$\color{SkyBlue}{0}$|
|$s_5$|$0$|$\color{SkyBlue}{0}$|
|$s_6$|$0$|$\color{SkyBlue}{0}$|
|$s_7$|$0$|$\color{SkyBlue}{0.5}$|
|$E$|$0$|$0$|

Let us follow the exact same sequence in the next episode. We see that all updates will have the value 0 up to and including $s_5 \rightarrow a_2 \rightarrow s_6$. But what does the update for $s_6 \rightarrow a_2 \rightarrow s_7$ look like?

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \cdot \left(r + \gamma \cdot \max_{a'} Q(s',a') - Q(s,a) \right)
$$

$$
Q(s_6,a_2) \leftarrow Q(s_6,a_2) + \alpha \cdot \left(r + \gamma \cdot \max_{a'} Q(s_7,a') - Q(s_6,a_2) \right)
$$

We read off $Q(s_6,a_2) = 0$  from our current Q-value table and also know that $r = 0$, $\alpha = 0.5$ and $\gamma = 0.9$:

$$
Q(s_6,a_2) \leftarrow 0 + 0.5 \cdot \left(0 + 0.9 \cdot \max_{a'} Q(s_7,a') - 0 \right)
$$

What is the value of $\max_{a'} Q(s_7,a')$? If we look in our current Q-value table, we see that $\max_{a'} Q(s_7,a_1) = 0$ and $\max_{a'} Q(s_7,a_2) = 0.5$. Therefore, $\max_{a'} Q(s_7,a') = 0.5$. If we substitute this value into our formula, we can complete the calculation:

$$
Q(s_6,a_2) \leftarrow 0 + 0.5 \cdot \left(0 + 0.9 \cdot 0.5 - 0 \right) = 0.225
$$

Now our Q-value table looks as follows:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad \,\,\, s_6 \,\,\, \quad$ | $\quad s_7 \quad$ |
|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$0 \mid \color{SkyBlue}{0.225}$|$ \, 0 \mid 0.5$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$ \quad 0 \quad$|

</div>
</div>

Or, in table form.

|State/Action|$a_1$|$a_2$|
|------------|-----|-----|
|$s_1$|$0$|$0$|
|$s_2$|$0$|$0$|
|$s_3$|$0$|$0$|
|$s_4$|$0$|$\color{SkyBlue}{0}$|
|$s_5$|$0$|$\color{SkyBlue}{0}$|
|$s_6$|$0$|$\color{SkyBlue}{0.225}$|
|$s_7$|$0$|$0.5$|
|$E$|$0$|$0$|

Now the agent is in state $s_7$ (the terminal state), where the action  $a_2$ is taken, after which the agent ends up in the virtual state $E$. We also carry out the update for this using the current values and obtain:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \cdot \left(r + \gamma \cdot \max_{a'} Q(s',a') - Q(s,a) \right)
$$

$$
Q(s_7,a_2) \leftarrow Q(s_7,a_2) + 0.5 \cdot \left(1 + 0.9 \cdot \max_{a'} Q(E,a') - Q(s_7,a_2) \right)
$$

$$
Q(s_7,a_2) \leftarrow 0.5 + 0.5 \cdot \left(1 + 0.9 \cdot 0 - 0.5 \right) = 0.75
$$

After the second episode, our Q-value table looks as follows:

<div style="display: flex; gap: 20px;">
<div>

||||||||
|-------|-------|-------|-------|-------|-------|-------|
| $\quad s_1 \quad$ | $\quad s_2 \quad$ | $\quad s_3 \quad$ | $\quad s_4 \quad$ | $\quad s_5 \quad$ | $\quad \,\,\, s_6 \,\,\, \quad$ | $\quad \, s_7 \, \quad$ |
|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, 0$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$ \, 0 \,\, \mid \,\, \color{SkyBlue}{0}$|$0 \mid \color{SkyBlue}{0.225}$|$0 \mid \color{SkyBlue}{0.75}$|

</div>
<div>

||
|-----|
|$\quad E \quad$|
|$ \quad 0 \quad$|

</div>
</div>

Or, in table form.

|State/Action|$a_1$|$a_2$|
|------------|-----|-----|
|$s_1$|$0$|$0$|
|$s_2$|$0$|$0$|
|$s_3$|$0$|$0$|
|$s_4$|$0$|$\color{SkyBlue}{0}$|
|$s_5$|$0$|$\color{SkyBlue}{0}$|
|$s_6$|$0$|$\color{SkyBlue}{0.225}$|
|$s_7$|$0$|$\color{SkyBlue}{0.75}$|
|$E$|$0$|$0$|

If we run enough episodes, where we vary the starting state and use an appropriate exploration strategy (e.g., $\epsilon$-greedy), we will eventually obtain the optimal Q-values, i.e., $Q^{*}(s,a)$.